In [3]:
import jax.numpy as jnp
from jax.typing import ArrayLike
from jax import Array
import jax
#jax.config.update('jax_array', True)
jax.__version__

'0.4.14'

In [4]:
def fn(x:ArrayLike, beta0:float, beta1:float, beta2:float, beta3:float, omega0:float, omega1:float, omega2:float, omega3:float)->Array:
  return beta3+omega3 * jnp.cos(beta2 + omega2 * jnp.exp(beta1 + omega1 * jnp.sin(beta0 + omega0 * x)))

def likelihood(x:ArrayLike, y:ArrayLike, beta0, beta1, beta2, beta3, omega0, omega1, omega2, omega3):
  diff = fn(x, beta0, beta1, beta2, beta3, omega0, omega1, omega2, omega3) - y ;
  return diff * diff ;

In [5]:
beta0 = 1.0; beta1 = 2.0; beta2 = -3.0; beta3 = 0.4;
omega0 = 0.1; omega1 = -0.4; omega2 = 2.0; omega3 = 3.0;
x = 2.3; y =2.0
l_i_func = likelihood(x,y,beta0,beta1,beta2,beta3,omega0,omega1,omega2,omega3)
print('l_i=%3.3f'%l_i_func)

l_i=0.139


In [6]:
dldbeta3_func = 2 * (beta3 +omega3 * jnp.cos(beta2 + omega2 * jnp.exp(beta1+omega1 * jnp.sin(beta0+omega0 * x)))-y)
dldomega0_func = -2 *(beta3 +omega3 * jnp.cos(beta2 + omega2 * jnp.exp(beta1+omega1 * jnp.sin(beta0+omega0 * x)))-y) * \
              omega1 * omega2 * omega3 * x * jnp.cos(beta0 + omega0 * x) * jnp.exp(beta1 +omega1 * jnp.sin(beta0 + omega0 * x)) *\
              jnp.sin(beta2 + omega2 * jnp.exp(beta1+ omega1* jnp.sin(beta0+omega0 * x)))

In [7]:
dldomega0_fd = (likelihood(x,y,beta0,beta1,beta2,beta3,omega0+0.00001,omega1,omega2,omega3)-likelihood(x,y,beta0,beta1,beta2,beta3,omega0,omega1,omega2,omega3))/0.00001

print('dydomega0: Function value = %3.3f, Finite difference value = %3.3f'%(dldomega0_func,dldomega0_fd))

dydomega0: Function value = 5.246, Finite difference value = 5.290


In [8]:
f0 = beta0 + omega0 * x
h1 = jnp.sin(f0)
f1 = beta1 + omega1 * h1
h2 = jnp.exp(f1)
f2 = beta2 + omega2 * h2
h3 = jnp.cos(f2)
f3 = beta3 + omega3 * h3
l_i = (f3-y) ** 2

In [9]:
# Let's check we got that right:
print("f0: true value = %3.3f, your value = %3.3f"%(1.230, f0))
print("h1: true value = %3.3f, your value = %3.3f"%(0.942, h1))
print("f1: true value = %3.3f, your value = %3.3f"%(1.623, f1))
print("h2: true value = %3.3f, your value = %3.3f"%(5.068, h2))
print("f2: true value = %3.3f, your value = %3.3f"%(7.137, f2))
print("h3: true value = %3.3f, your value = %3.3f"%(0.657, h3))
print("f3: true value = %3.3f, your value = %3.3f"%(2.372, f3))
print("like original = %3.3f, like from forward pass = %3.3f"%(l_i_func, l_i))


f0: true value = 1.230, your value = 1.230
h1: true value = 0.942, your value = 0.942
f1: true value = 1.623, your value = 1.623
h2: true value = 5.068, your value = 5.068
f2: true value = 7.137, your value = 7.137
h3: true value = 0.657, your value = 0.657
f3: true value = 2.372, your value = 2.372
like original = 0.139, like from forward pass = 0.139


In [19]:
# TODO -- Compute the derivatives of the output with respect
# to the intermediate computations h_k and f_k (i.e, run the backward pass)
# I've done the first two for you.  You replace the code below:
dldf3 = 2* (f3 - y)
dldh3 = omega3 * dldf3
# Replace the code below
dldf2 =-jnp.sin(f2)*dldh3
dldh2 = omega2 * dldf2
dldf1 = jnp.exp(f1) * dldh2
dldh1 = omega1* dldf1
dldf0 = jnp.cos(f0)*dldh1


In [20]:
# Let's check we got that right
print("dldf3: true value = %3.3f, your value = %3.3f"%(0.745, dldf3))
print("dldh3: true value = %3.3f, your value = %3.3f"%(2.234, dldh3))
print("dldf2: true value = %3.3f, your value = %3.3f"%(-1.683, dldf2))
print("dldh2: true value = %3.3f, your value = %3.3f"%(-3.366, dldh2))
print("dldf1: true value = %3.3f, your value = %3.3f"%(-17.060, dldf1))
print("dldh1: true value = %3.3f, your value = %3.3f"%(6.824, dldh1))
print("dldf0: true value = %3.3f, your value = %3.3f"%(2.281, dldf0))

dldf3: true value = 0.745, your value = 0.745
dldh3: true value = 2.234, your value = 2.234
dldf2: true value = -1.683, your value = -1.683
dldh2: true value = -3.366, your value = -3.366
dldf1: true value = -17.060, your value = -17.060
dldh1: true value = 6.824, your value = 6.824
dldf0: true value = 2.281, your value = 2.281


In [21]:
dldbeta3 = dldf3
dldomega3 = h3 * dldf3
dldbeta2 = dldf2
dldomega2 = h2 * dldf2
dldbeta1 = dldf1
dldomega1 = h1* dldf1
dldbeta0 = dldf0
dldomega0 = x * dldf0

In [22]:
# Let's check we got them right
print('dldbeta3: Your value = %3.3f, True value = %3.3f'%(dldbeta3, 0.745))
print('dldomega3: Your value = %3.3f, True value = %3.3f'%(dldomega3, 0.489))
print('dldbeta2: Your value = %3.3f, True value = %3.3f'%(dldbeta2, -1.683))
print('dldomega2: Your value = %3.3f, True value = %3.3f'%(dldomega2, -8.530))
print('dldbeta1: Your value = %3.3f, True value = %3.3f'%(dldbeta1, -17.060))
print('dldomega1: Your value = %3.3f, True value = %3.3f'%(dldomega1, -16.079))
print('dldbeta0: Your value = %3.3f, True value = %3.3f'%(dldbeta0, 2.281))
print('dldomega0: Your value = %3.3f, Function value = %3.3f, Finite difference value = %3.3f'%(dldomega0, dldomega0_func, dldomega0_fd))

dldbeta3: Your value = 0.745, True value = 0.745
dldomega3: Your value = 0.489, True value = 0.489
dldbeta2: Your value = -1.683, True value = -1.683
dldomega2: Your value = -8.530, True value = -8.530
dldbeta1: Your value = -17.060, True value = -17.060
dldomega1: Your value = -16.079, True value = -16.079
dldbeta0: Your value = 2.281, True value = 2.281
dldomega0: Your value = 5.246, Function value = 5.246, Finite difference value = 5.290
